In [3]:
#get processed data

import pandas as pd

lut = pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/lutein.csv?raw=true")

sug = pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/sugars.csv?raw=true")

wat=pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/water.csv?raw=true")

#dropping Name column
lut=lut.drop('name',axis=1)
sug=sug.drop('name',axis=1)
wat=wat.drop('name',axis=1)


In [5]:
#create arrays with only the rows that have NaN to measure performance on at the end
import numpy as np

lutmarnan2=lut[np.isnan(lut['lutein_zeaxanthin_mar'])]
lutmarnan=lutmarnan2.drop(['lutein_zeaxanthin_mcar','lutein_zeaxanthin'],axis=1)

lutmcarnan2=lut[np.isnan(lut['lutein_zeaxanthin_mcar'])]
lutmcarnan=lutmcarnan2.drop(['lutein_zeaxanthin_mar','lutein_zeaxanthin'],axis=1)

sugmarnan2=sug[np.isnan(sug['sugars_mar'])]
sugmarnan=sugmarnan2.drop(['sugars_mcar','sugars'],axis=1)

sugmcarnan2=sug[np.isnan(sug['sugars_mcar'])]
sugmcarnan=sugmcarnan2.drop(['sugars_mar','sugars'],axis=1)

watmarnan2=wat[np.isnan(wat['water_mar'])]
watmarnan=watmarnan2.drop(['water_mcar','water'],axis=1)

watmcarnan2=wat[np.isnan(wat['water_mcar'])]
watmcarnan=watmcarnan2.drop(['water_mar','water'],axis=1)

lutmarnan=lutmarnan.to_numpy()
lutmcarnan=lutmcarnan.to_numpy()

sugmarnan=sugmarnan.to_numpy()
sugmcarnan=sugmcarnan.to_numpy()

watmarnan=watmarnan.to_numpy()
watmcarnan=watmcarnan.to_numpy()


In [6]:
#replace NaN using MICE algorithm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


imp = IterativeImputer(max_iter=10, random_state=2023)

#lutein mcar
#Need to drop lutein mar and true lutein columns
lutmcar=lut.copy()
lutmcar = lutmcar[lutmcar['lutein_zeaxanthin_mcar'].notnull()]
lutmcar=lutmcar.drop(['lutein_zeaxanthin_mar','lutein_zeaxanthin'],axis=1)
lutmcarn=lutmcar.to_numpy()
imp.fit(lutmcarn)
lutmcar2=imp.transform(lutmcarnan)

lutmcarimp = pd.DataFrame(lutmcar2, columns = lutmcar.columns)

#lutein mar
#Need to drop lutein mcar and true lutein columns
lutmar=lut.copy()
lutmar = lutmar[lutmar['lutein_zeaxanthin_mar'].notnull()]
lutmar=lutmar.drop(['lutein_zeaxanthin_mcar','lutein_zeaxanthin'],axis=1)
lutmarn=lutmar.to_numpy()
imp.fit(lutmarn)
lutmar2=imp.transform(lutmarnan)

lutmarimp = pd.DataFrame(lutmar2, columns = lutmar.columns)

#sugars mcar
#Need to drop sugar mar and true sugar columns
sugmcar=sug.copy()
sugmcar = sugmcar[sugmcar['sugars_mcar'].notnull()]
sugmcar=sugmcar.drop(['sugars_mar','sugars'],axis=1)
sugmcarn=sugmcar.to_numpy()
imp.fit(sugmcarn)
sugmcar2=imp.transform(sugmcarnan)
sugmcarimp = pd.DataFrame(sugmcar2, columns = sugmcar.columns)


#sugars mar
#Need to drop sugar mcar and true sugar columns
sugmar=sug.copy()
sugmar = sugmar[sugmar['sugars_mar'].notnull()]
sugmar=sugmar.drop(['sugars_mcar','sugars'],axis=1)
sugmarn=sugmar.to_numpy()
imp.fit(sugmarn)
sugmar2=imp.transform(sugmarnan)
sugmarimp = pd.DataFrame(sugmar2, columns = sugmar.columns)


#water mcar
#Need to drop water mar and true water columns
watmcar=wat.copy()
watmcar = watmcar[watmcar['water_mcar'].notnull()]
watmcar=watmcar.drop(['water_mar','water'],axis=1)
watmcarn=watmcar.to_numpy()
imp.fit(watmcarn)
watmcar2=imp.transform(watmcarnan)
watmcarimp = pd.DataFrame(watmcar2, columns = watmcar.columns)


#water mar
#Need to drop water mcar and true water columns
watmar=wat.copy()
watmar = watmar[watmar['water_mar'].notnull()]
watmar=watmar.drop(['water_mcar','water'],axis=1)
watmarn=watmar.to_numpy()
imp.fit(watmarn)
watmar2=imp.transform(watmarnan)
watmarimp = pd.DataFrame(watmar2, columns = watmar.columns)


In [7]:
#adding back the column with true values


lutmcarimp['true_lut'] = lutmcarnan2['lutein_zeaxanthin'].values
lutmarimp['true_lut'] = lutmarnan2['lutein_zeaxanthin'].values

sugmcarimp['true_sug'] = sugmcarnan2['sugars'].values
sugmarimp['true_sug'] = sugmarnan2['sugars'].values

watmcarimp['true_wat'] = watmcarnan2['water'].values
watmarimp['true_wat'] = watmarnan2['water'].values

In [8]:
#calculating rmse for each columns and missing type
import math
import numpy as np


meth='MICE'
results=[]

#lutein mcar
dat='lutein'
dt=lutmcarimp

dt['sqrt']=(dt['true_lut']-dt['lutein_zeaxanthin_mcar'])**2
lutmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",lutmcarrmse)
results.append([dat,'mcar',meth,lutmcarrmse])

#lutein mar
dt=lutmarimp

dt['sqrt']=(dt['true_lut']-dt['lutein_zeaxanthin_mar'])**2
lutmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",lutmarrmse)
results.append([dat,'mar',meth,lutmarrmse])

#sugars mcar
dat='sugars'
dt=sugmcarimp

dt['sqrt']=(dt['true_sug']-dt['sugars_mcar'])**2
sugmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",sugmcarrmse)
results.append([dat,'mcar',meth,sugmcarrmse])

#sugars mar
dt=sugmarimp

dt['sqrt']=(dt['true_sug']-dt['sugars_mar'])**2
sugmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",sugmarrmse)
results.append([dat,'mar',meth,sugmarrmse])

#water mcar
dat='water'
dt=watmcarimp

dt['sqrt']=(dt['true_wat']-dt['water_mcar'])**2
watmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",watmcarrmse)
results.append([dat,'mcar',meth,watmcarrmse])

#water mcar
dt=watmarimp

dt['sqrt']=(dt['true_wat']-dt['water_mar'])**2
watmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",watmarrmse)
results.append([dat,'mar',meth,watmarrmse])

print(results)

RMSE: 766.8691658682543
RMSE: 1036.1884615241652
RMSE: 9.756056972257303
RMSE: 11.728405161596655
RMSE: 0.7995658956461243
RMSE: 0.8240912063564731
[['lutein', 'mcar', 'MICE', 766.8691658682543], ['lutein', 'mar', 'MICE', 1036.1884615241652], ['sugars', 'mcar', 'MICE', 9.756056972257303], ['sugars', 'mar', 'MICE', 11.728405161596655], ['water', 'mcar', 'MICE', 0.7995658956461243], ['water', 'mar', 'MICE', 0.8240912063564731]]


In [9]:
res = pd.DataFrame(results, columns = ['Column', 'Missing type','Imputation method','RMSE'])
print(res)

   Column Missing type Imputation method         RMSE
0  lutein         mcar              MICE   766.869166
1  lutein          mar              MICE  1036.188462
2  sugars         mcar              MICE     9.756057
3  sugars          mar              MICE    11.728405
4   water         mcar              MICE     0.799566
5   water          mar              MICE     0.824091
